In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt


sys.path.append('../')

from BGS import BGS
from FileEmcee import FileEmcee
from EmceeRun import EmceeRun
from Plotter import Plotter
from VMaxDensity import VmaxDensity
from ZSchechterModel import ZSchechterModel
from Fileh5 import Fileh

In [2]:
bgs = BGS('BGS_ANY_full.provabgs.lite.hdf5', is_bgs_bright=True)
emcee_file = Fileh5('bin_tests_runs', 'a3deeper')

In [3]:
emcee_file.get_file()

Multiple files with that <info_file> 
 [(0, '/Users/federico/OneDrive - University College London/PhD/PhD_project/bgs_psmf/emcee_runs/bin_tests_runs/ZSchechter_50w_600step_.01_0.4_a3deeper.h5'), (1, '/Users/federico/OneDrive - University College London/PhD/PhD_project/bgs_psmf/emcee_runs/bin_tests_runs/ZSchechter_20w_400step_0.0_0.2_a3deeper.h5')] 
 - Tell me the index of the file you want to analyse:  1


'/Users/federico/OneDrive - University College London/PhD/PhD_project/bgs_psmf/emcee_runs/bin_tests_runs/ZSchechter_20w_400step_0.0_0.2_a3deeper.h5'

In [ ]:
bgs = BGS('BGS_ANY_full.provabgs.lite.hdf5', is_bgs_bright=True)

emcee_file_02 = FileEmcee(z_dependence=True, bin_test=True, info_file='0.0_0.2_a3deeper')
emcee_file_04 = FileEmcee(z_dependence=True, bin_test=True, info_file='.01_0.4_a3deeper')
emcee_file_new = FileEmcee(z_dependence=True, bin_test=True, info_file='new_prior')

emcee_run_02 = EmceeRun(emcee_file_02)
emcee_run_04 = EmceeRun(emcee_file_04)
emcee_run_new = EmceeRun(emcee_file_new)

In [ ]:
plotter02 = Plotter(bgs, emcee_run_02)
plotter04 = Plotter(bgs, emcee_run_04)
plotter_new = Plotter(bgs, emcee_run_new)


In [ ]:
plotter02.plot_emcee_samples()

In [ ]:
plotter04.plot_emcee_samples()

In [ ]:
plotter_new.plot_emcee_samples()

In [ ]:
emcee_run_02.set_best_params(discard=50)
emcee_run_04.set_best_params(discard=150)
emcee_run_new.set_best_params(discard=70)

In [ ]:
plotter_new.plot_emcee_corner(emcee_run_new.flat_samples, emcee_run_new.labels4)

In [ ]:
# best_params = emcee_run_02.get_best_params()
best_params_a3deeper = emcee_run_04.get_best_params()
best_params = emcee_run_new.get_best_params()

In [ ]:
z = np.linspace(0.01,1,100)

plt.figure(figsize=(6,6))
plt.subplot(1,2,1)
plt.plot(z, best_params[0] + best_params[1]*z, label=r'fit up to $z=0.2$')
plt.plot(z, best_params_a3deeper[0] + best_params_a3deeper[1]*z, label='fit up to $z=0.4$')
plt.xlim(0,0.41)
plt.legend()

plt.subplot(1,2,2)
plt.plot(z, best_params[2] + best_params[3]*z, label=r'fit up to $z=0.2$')
plt.plot(z, best_params_a3deeper[2] + best_params_a3deeper[3]*z, label='fit up to $z=0.4$')
plt.xlim(0,0.41)
plt.legend()

In [ ]:
vmax_hist = VmaxDensity(bgs.file, is_bgs_bright=True)

In [ ]:
zmins = np.array([0.01 + 0.04 * i for i in range(4)])
zmaxs = np.array([0.05 + 0.04 * i for i in range(4)])

x_lin_ext = np.linspace(7, 14, 100)

fig = plt.figure(figsize=(15,30))

plot_params_ext = {'color': 'orange', 'linestyle':'--', 'linewidth':0.7}
plot_params = {'color':'orange', 'linestyle':'-', 'label':'up to 0.2'}

plot_params_ext2 = {'color': 'green', 'linestyle':'--', 'linewidth':0.7, 'label':'a3'}
plot_params2 = {'color':'green', 'linestyle':'-'}

# plot_params_ext3 = {'color': 'red', 'linestyle':'--', 'linewidth':0.7}
# plot_params3 = {'color':'red', 'linestyle':'-'}

for i, zmin, zmax, z0 in zip(range(9), zmins, zmaxs, zmins + (zmaxs - zmins)/2.):
    plt.subplot(5,2,i+1)
    h, b, _h, _b = vmax_hist.histogram_norm(zmin, zmax)
    z_smf = ZSchechterModel(bgs, zmin, zmax, z0)
    
    n = z_smf.normalisation(best_params)


    x_lin = np.linspace(z_smf.mlim.min(), 14, 100)

    plotter04.plot_vmax_hist(h, b, _h, _b, **{'label':f'${zmin:.2f}< z <{zmax:.2f}$'})
    Plotter.plot_zschechter(x_lin, z0, n, best_params, **plot_params)
    Plotter.plot_zschechter(x_lin_ext, z0, n, best_params, **plot_params_ext)
    Plotter.plot_zschechter(x_lin, z0, n, best_params_a3deeper, **plot_params2)
    Plotter.plot_zschechter(x_lin_ext, z0, n, best_params_a3deeper, **plot_params_ext2)


    # inds = np.random.randint(len(emcee_run.flat_samples), size=100)
    # for ind in inds:
    #     z_smf_err = ZSchechterModel(bgs, zmin, zmax, z0)
    
    #     n_err = z_smf_err.normalisation(emcee_run.flat_samples[ind])
    #     Plotter.plot_zschechter(x_lin_ext, z0, n_err, emcee_run.flat_samples[ind], **{'alpha':0.2})

    
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.legend(loc='best')
plt.show()    